# Jupyter Snake

In [ ]:
import os

from algorithms import *
from snake_environment import *
from states_bracket import *
from epsilon_scheduler import *
from utils import *

In [ ]:
current_path = os.getcwd()
models_path = current_path + "/models/"

In [ ]:
# Bracketer
bracketer = NeighPlusFoodDirectionBracket(neigh="V", radius=1)
# General Settings 
gamma = 0.99
lr_v = 0.15
n_episodes = 25000
epsilon_schedule = LinearEpsilonDecay(eps = 1, coefficient=0.999, minimum=0.15)

## QLearning

In [ ]:
# Environment
env = SnakeEnv(render_mode="nonhuman", max_step=1000)
Q_p = QLearning(env.action_space.n, gamma=gamma, lr_v=lr_v)
Q_p.learning(env, epsilon_schedule, n_episodes, bracketer)

In [ ]:
name_specs = f'linear from 1 to 015 with 0999 as coefficient vn1 plus fd 1000 max iterations'
model_path = f"./models/QLearning gamma {gamma} lr {lr_v} epsilon {name_specs} 2nd"

In [ ]:
Q_p.save(f"{model_path}")

In [ ]:
Q_p.upload(f"{model_path}")

In [ ]:
env = SnakeEnv(render_mode="human", max_step=1000)
Q_p.play(env, bracketer)

In [ ]:
Q_p.print_q_values(bracketer)

## SARSA

In [ ]:
# Environment
epsilon_schedule = LinearEpsilonDecay(eps = 1, coefficient=0.9999, minimum=0.30)
env = SnakeEnv(render_mode="nonhuman", max_step=1000)
SARSA_p = SARSA(env.action_space.n, gamma=gamma, lr_v=lr_v)
bracketer = FoodDirectionBracket()
SARSA_p.learning(env, epsilon_schedule, n_episodes = 50000, bracketer = bracketer)

In [ ]:
name_specs = f'linear from 1 to 030 with 09999 as coefficient vn1 plus fd 1000 max iterations'
model_path = f"{models_path}SARSA gamma {gamma} lr {lr_v} epsilon {name_specs}"
SARSA_p.save(f"{model_path}")

In [ ]:
SARSA_p.upload(f"{model_path}")

In [ ]:
env = SnakeEnv(render_mode="human", max_step=1000)
SARSA_p.play(env, bracketer)

In [ ]:
SARSA_p.print_q_values(bracketer)

## Double Deep Q-Learning


In [ ]:
BATCH_SIZE = 128
MEMORY_SIZE = 10000
TARGET_UPDATE_FREQ = 200

# Bracketer
bracketer = NeighPlusFoodDirectionBracket(neigh="V", radius=1)
# General Settings
gamma = 0.95
lr_v = 0.001
epsilon = 0.1
n_episodes = 5001
epsilon_schedule = LinearEpsilonDecay(eps = 1, coefficient=0.999, minimum=0.05)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'xpu' if torch.xpu.is_available() else device
print(f"Using device: {device}")

# Environment
env = SnakeEnv(render_mode="nonhuman")
state_dim = bracketer.get_state_dim()

ddqn = DeepDoubleQLearning(
    env.action_space.n,
    state_dim=state_dim,
    gamma=gamma,
    lr_v=lr_v,
    batch_size=BATCH_SIZE,
    memory_size=MEMORY_SIZE,
    target_update_freq=TARGET_UPDATE_FREQ,
    device=device
)

In [ ]:
env = SnakeEnv(render_mode="nonhuman")
ddqn.learning(env, epsilon_schedule, n_episodes, bracketer)

In [ ]:
model_path = f"{models_path}DDQN gamma {gamma} lr {lr_v} epsilon {epsilon} episodes {n_episodes} bracketer {bracketer.__class__.__name__} "

In [ ]:
ddqn.save(path=f'{model_path}')

In [ ]:
ddqn.upload(model_path)

In [ ]:
env = SnakeEnv(render_mode='human', max_step=2000)
ddqn.play(env, bracketer)

In [ ]:
get_model_average_score(model_name='DDQN', action_space=env.action_space.n, gamma=gamma, lr_v=lr_v, model_path=model_path, bracketer=bracketer, num_episodes=1000, render_mode='nonhuman', state_dim=state_dim, batch_size=BATCH_SIZE, memory_size=MEMORY_SIZE, target_update_freq=TARGET_UPDATE_FREQ, device=device)

## Atari-like DDQL

In [8]:
BATCH_SIZE = 32
MEMORY_SIZE = 10000
TARGET_UPDATE_FREQ = 200

# Bracketer
width = 20
height = 20
bracketer = FullGrid(width=width, height=height)
# General Settings
gamma = 0.95
lr_v = 0.001
epsilon = 0.1
n_episodes = 20001
epsilon_schedule = LinearEpsilonDecay(eps = 1, coefficient=0.9999, minimum=0.05)
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'xpu' if torch.xpu.is_available() else device
print(f"Using device: {device}")

# Environment
env = SnakeEnv(render_mode="nonhuman")
state_dim = bracketer.get_state_dim()

atari = AtariDeepQLearning(
    env.action_space.n,
    gamma=gamma,
    lr_v=lr_v,
    batch_size=BATCH_SIZE,
    memory_size=MEMORY_SIZE,
    target_update_freq=TARGET_UPDATE_FREQ,
    device=device,
    width=width,
    height=height,
    n_layers=1
)

Using device: xpu


In [ ]:
env = SnakeEnv(render_mode="nonhuman")
atari.learning(env, epsilon_schedule, n_episodes, bracketer)

In [ ]:
model_path = f"{models_path}AtariDQL gamma {gamma} lr {lr_v} epsilon {epsilon} episodes {n_episodes} bracketer {bracketer.__class__.__name__} "

In [ ]:
atari.save(path=f'{model_path}')

In [ ]:
atari.upload(model_path)

In [ ]:
env = SnakeEnv(render_mode='human', max_step=2000)
atari.play(env, bracketer)

In [ ]:
get_model_average_score(model_name='AtariDDQL', action_space=env.action_space.n, gamma=gamma, lr_v=lr_v, model_path=model_path, bracketer=bracketer, num_episodes=1000, render_mode='nonhuman', batch_size=BATCH_SIZE, memory_size=MEMORY_SIZE, target_update_freq=TARGET_UPDATE_FREQ, device=device, height=height, width=width, n_layers = 1)

## Monte Carlo

In [ ]:
env = SnakeEnv(render_mode="nonhuman")
n_episodes = 5000
bracketer = NeighPlusFoodDirectionBracket(neigh="V", radius=1)
MC = Montecarlo(env.action_space.n, gamma=gamma, lr_v=lr_v)

In [ ]:
MC.learning(env, epsilon_schedule, n_episodes, bracketer)

In [ ]:
path = "./models/"
MC.save(f"{path}MC gamma {gamma} lr {lr_v} epsilon {epsilon_schedule} episodes {n_episodes} bracketer {bracketer.__class__.__name__}")

In [ ]:
env = SnakeEnv(render_mode="human")
# MC.upload(f"{path}MC gamma {gamma} lr {lr_v} epsilon {epsilon_schedule} episodes {n_episodes} bracketer {bracketer.__class__.__name__}")

In [ ]:
MC.play(env, bracketer)

In [ ]:
model_path = f"{path}MC gamma {gamma} lr {lr_v} epsilon {epsilon_schedule} episodes {n_episodes} bracketer {bracketer.__class__.__name__}"

get_model_average_score(model_name='MC', action_space=env.action_space.n, gamma=gamma, lr_v=lr_v, model_path=model_path, bracketer=bracketer, num_episodes=1000, render_mode='nonhuman')